# 📊 Data Analyst Portfolio (Interactive)
Notebook interaktif ini berisi analisis penjualan, evaluasi promosi, dan dashboard capaian tim marketing.

**Fitur utama:**
- Laporan Harian / Mingguan / Bulanan (omzet & profit) + ekspor Excel
- Analisis produk (Pareto) & kategori
- Evaluasi campaign (ROI, baseline vs actual)
- Dashboard interaktif untuk memilih bulan / region / tim
- Storytelling otomatis (insight tertulis berdasarkan data)

---
**Petunjuk:** Jalankan sel dari atas ke bawah. Untuk interaktivitas gunakan widget yang muncul di setiap seksi.

In [ ]:
# ====== 0. Imports & Settings ======
import os
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import display, Markdown
import ipywidgets as widgets
pd.set_option('display.max_columns', None)

def fmt_currency(x):
    try: return f"Rp {x:,.0f}".replace(',', '.')
    except: return x

def add_fmt_currency(df, cols):
    df = df.copy()
    for c in cols:
        if c in df.columns:
            df[f'{c}_fmt'] = df[c].apply(fmt_currency)
    return df

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Data Preparation
Memuat dataset yang telah disediakan perusahaan. Catatan: pada data transaksi, campaign_id sering kosong karena transaksi reguler yang tidak terkait kampanye. Kita akan tandai sebagai "No Campaign" untuk keperluan agregasi.

In [ ]:
# ====== 1. Load datasets ======
SALES_PATH = "/content/drive/MyDrive/DA_portfolio/data/sales_transactions.csv"
PRODUCTS_PATH = "/content/drive/MyDrive/DA_portfolio/data/products.csv"
CAMPAIGN_RESULTS_PATH = "/content/drive/MyDrive/DA_portfolio/data/campaign_results.csv"
PROMO_PROPOSALS_PATH = "/content/drive/MyDrive/DA_portfolio/data/promo_proposals.csv"
TEAM_PATH = "/content/drive/MyDrive/DA_portfolio/data/team_marketing.csv"
TARGETS_PATH = "/content/drive/MyDrive/DA_portfolio/data/team_targets_monthly.csv"

sales = pd.read_csv(SALES_PATH)
products = pd.read_csv(PRODUCTS_PATH)
campaign_results = pd.read_csv(CAMPAIGN_RESULTS_PATH)
promos = pd.read_csv(PROMO_PROPOSALS_PATH)
team = pd.read_csv(TEAM_PATH)
targets = pd.read_csv(TARGETS_PATH)

# Basic clean
sales['date'] = pd.to_datetime(sales['date'])
targets['month'] = pd.to_datetime(targets['month'])
sales['campaign_id'] = sales['campaign_id'].fillna('No Campaign')
if 'gross_revenue' not in sales.columns and {'unit_price','quantity'}.issubset(sales.columns):
    sales['gross_revenue'] = sales['unit_price'] * sales['quantity']
if 'gross_profit' not in sales.columns and {'gross_revenue','cost_of_goods'}.issubset(sales.columns):
    sales['gross_profit'] = sales['gross_revenue'] - sales['cost_of_goods']

print('Loaded: sales', len(sales), 'rows; products', len(products), 'rows')


Loaded: sales 27108 rows; products 18 rows


In [ ]:
# ====== Helper functions ======
def aggregate_timeframe(df, date_col, value_cols, freq='D'):
    g = (df.set_index(date_col).resample(freq)[value_cols].sum().reset_index())
    return g

def growth_rate(series):
    # simple month-over-month growth for series with datetime index
    s = series.copy().reset_index(drop=True)
    return s.pct_change()*100

## Laporan Harian / Mingguan / Bulanan (Omzet & Profit)
Bagian ini menghasilkan tiga tabel ringkas (harian, mingguan, bulanan) dan visual tren omzet bulanan untuk kebutuhan Business Development & manajemen.

In [ ]:
# prepare aggregates
value_cols = ['gross_revenue','gross_profit']
daily = aggregate_timeframe(sales, 'date', value_cols, freq='D')
weekly = aggregate_timeframe(sales, 'date', value_cols, freq='W')
monthly = aggregate_timeframe(sales, 'date', value_cols, freq='M')

# widget: pilih level
level_sel = widgets.Dropdown(options=['Daily','Weekly','Monthly'], value='Daily', description='Periode:')
out = widgets.Output()

def show_table(level):
    out.clear_output()
    if level=='Daily': df = daily
    elif level=='Weekly': df = weekly
    else: df = monthly
    with out:
        display(df.head(15))

widgets.interact(show_table, level=level_sel)
display(out)

/tmp/ipython-input-1408107259.py:3: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  g = (df.set_index(date_col).resample(freq)[value_cols].sum().reset_index())


interactive(children=(Dropdown(description='Periode:', options=('Daily', 'Weekly', 'Monthly'), value='Daily'),…

Output()

In [ ]:
# Grafik omzet
fig = px.line(daily, x='date', y='gross_revenue', title='Trend Omzet Harian')
fig.show()

fig = px.line(weekly, x='date', y='gross_revenue', title='Trend Omzet Mingguan')
fig.show()

fig = px.line(monthly, x='date', y='gross_revenue', title='Trend Omzet Bulanan')
fig.show()

## Analisis Produk — Pareto & Kategori (Interaktif)
Fokus ke kontribusi **kategori** dan **produk unggulan** agar strategi promosi & persediaan lebih tepat.

In [ ]:
sales_prod = sales.merge(products, on='product_id', how='left')

# Dropdown kategori
cats = ['All'] + sorted(sales_prod['category'].dropna().unique().tolist())
cat_dropdown = widgets.Dropdown(options=cats, value='All', description='Kategori:')

# Output hanya untuk tabel
out_table = widgets.Output()

def show_table(category):
    out_table.clear_output()
    df = sales_prod.copy()
    if category != 'All':
        df = df[df['category'] == category]
    prod_agg = df.groupby('product_name')['gross_revenue'].sum().reset_index().sort_values('gross_revenue', ascending=False)
    top = prod_agg.head(10)
    with out_table:
        display(top)

widgets.interact(show_table, category=cat_dropdown)
display(out_table)

interactive(children=(Dropdown(description='Kategori:', options=('All', 'Beverage', 'Dairy', 'Grocery', 'Snack…

Output()

In [ ]:
if products is not None and 'category' in products.columns:
    merged = sales.merge(products[['product_id','product_name','category']], on='product_id', how='left')
else:
    merged = sales.copy(); merged['product_name'] = merged.get('product_id'); merged['category'] = 'Unknown'

# Kategori
category_sales = merged.groupby('category', as_index=False)['gross_revenue'].sum().sort_values('gross_revenue', ascending=False)

fig = px.bar(category_sales, x='category', y='gross_revenue', title='Revenue per Kategori', text='gross_revenue')
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.show()

# Distribusi penjualan per kategori produk
category_sales['percentage'] = (category_sales['gross_revenue'] / category_sales['gross_revenue'].sum()) * 100

fig = px.pie(category_sales,
             names="category",
             values="gross_revenue",
             title="Distribusi Penjualan per Kategori Produk",
             hole=0.3)

fig.update_traces(textinfo="percent+label")  # tampilkan persen + nama kategori
fig.show()

# Pareto top 10 produk
# Treemap kontribusi produk (Top 10)
prod_sales = merged.groupby(['product_id','product_name'], as_index=False)['gross_revenue'].sum()
top10 = prod_sales.sort_values('gross_revenue', ascending=False).head(10)
top10['perc'] = top10['gross_revenue'] / top10['gross_revenue'].sum() * 100

fig = px.treemap(top10, path=['product_name'], values='gross_revenue', title='Kontribusi Produk (Top 10) — Treemap')
fig.update_traces(textinfo="label+value+percent parent")
fig.show()

## Promotion & Campaign Analysis (Interaktif)

In [ ]:
camp_ids = ['All'] + campaign_results['campaign_id'].astype(str).tolist()
camp_dropdown = widgets.Dropdown(options=camp_ids, value='All', description='Campaign:')
out3 = widgets.Output()

def show_campaign_table(camp):
    out3.clear_output()
    df = campaign_results.copy()
    if camp != 'All':
        df = df[df['campaign_id'].astype(str) == camp]
    with out3:
        display(df[['campaign_id','channel','budget','baseline_revenue','actual_revenue','roi']])

widgets.interact(show_campaign_table, camp=camp_dropdown)
display(out3)

interactive(children=(Dropdown(description='Campaign:', options=('All', 'P001', 'P004', 'P005', 'P009', 'P010'…

Output()

In [ ]:
def plot_campaign(camp='All', df_campaign=None):
    if df_campaign is None:
        df = campaign_results.copy()
    else:
        df = df_campaign.copy()

    if camp != 'All':
        df = df[df['campaign_id'].astype(str) == camp]

    if df.shape[0] == 0:
        print("Tidak ada data untuk campaign ini.")
        return

    # Grafik ROI per Campaign
    fig = px.bar(df, x='campaign_id', y='roi', color='channel', title=f'ROI Campaign — {camp}')
    fig.show()

    # Insight
    best = df.loc[df['roi'].idxmax()]
    worst = df.loc[df['roi'].idxmin()]
    print(f"📌 Campaign terbaik (ROI): {best['campaign_id']} — ROI {best['roi']:.2f}")
    print(f"📌 Campaign terendah (ROI): {worst['campaign_id']} — ROI {worst['roi']:.2f}")

# Contoh panggil
plot_campaign('All')   # Semua campaign


📌 Campaign terbaik (ROI): P001 — ROI -1.00
📌 Campaign terendah (ROI): P001 — ROI -1.00


## Dashboard Tim Marketing
Menggabungkan **realisasi** dari transaksi dengan **target** per anggota tim untuk memantau capaian bulanan dan progres per region.

In [ ]:
# ====== Dashboard Tim: Tabel Interaktif ======

month_dropdown = widgets.Dropdown(options=months, value=months[-1] if months else None, description='Bulan:')
region_opts = ['All'] + sorted(prog['region'].dropna().unique().tolist())
region_dropdown = widgets.Dropdown(options=region_opts, value='All', description='Region:')
out4 = widgets.Output()

def show_team_table(month, region):
    out4.clear_output()
    df = prog[prog['month']==month].copy()
    if region != 'All':
        df = df[df['region']==region]
    with out4:
        display(df[['team_member_id','name','team','region',
                    'target_omset','gross_revenue','achievement_pct']]
                .sort_values('achievement_pct', ascending=False))

widgets.interact(show_team_table, month=month_dropdown, region=region_dropdown)
display(out4)


interactive(children=(Dropdown(description='Bulan:', index=19, options=(Timestamp('2024-01-01 00:00:00'), Time…

Output()

In [ ]:
def plot_team_dashboard(month, region='All', df_prog=None):
    if df_prog is None:
        df = prog.copy()
    else:
        df = df_prog.copy()

    df = df[df['month'] == month].copy()
    if region != 'All':
        df = df[df['region'] == region]

    if df.shape[0] == 0:
        print("Tidak ada data untuk filter ini.")
        return

    # Grafik capaian target
    fig = px.bar(df, x='name', y='achievement_pct', color='team',
                 title=f'Capaian Target — {month:%B %Y}')
    fig.update_yaxes(ticksuffix='%')
    fig.show()

    # Insight otomatis
    top = df.loc[df['achievement_pct'].idxmax()]
    bot = df.loc[df['achievement_pct'].idxmin()]
    avg = df['achievement_pct'].mean()
    display(Markdown(
        f"**Insight otomatis:**\n"
        f"- Top performer: **{top['name']}** ({top['achievement_pct']:.1f}%)\n"
        f"- Bottom performer: **{bot['name']}** ({bot['achievement_pct']:.1f}%)\n"
        f"- Rata-rata capaian tim: **{avg:.1f}%**"
    ))

# Contoh panggil:
plot_team_dashboard(months[-1], 'All')   # bulan terakhir, semua region
# atau
plot_team_dashboard(months[-1], 'RegionName')


**Insight otomatis:**
- Top performer: **Eka** (3.5%)
- Bottom performer: **Budi** (2.1%)
- Rata-rata capaian tim: **2.7%**

Tidak ada data untuk filter ini.


## Ekspor Laporan ke Excel (Daily/Weekly/Monthly & Team Progress)

In [ ]:
EXPORT_PATH = 'interactive_marketing_reports.xlsx'
with pd.ExcelWriter(EXPORT_PATH, engine='openpyxl') as writer:
    add_fmt_currency(daily, ['gross_revenue','gross_profit']).to_excel(writer, sheet_name='Daily_Report', index=False)
    add_fmt_currency(weekly, ['gross_revenue','gross_profit']).to_excel(writer, sheet_name='Weekly_Report', index=False)
    add_fmt_currency(monthly, ['gross_revenue','gross_profit']).to_excel(writer, sheet_name='Monthly_Report', index=False)
    prog[prog['month']==(month_dropdown.value if month_dropdown.value is not None else prog['month'].max())].to_excel(writer, sheet_name='Team_Progress_Latest', index=False)

    # summary region
    region_summary = prog[prog['month']==(month_dropdown.value if month_dropdown.value is not None else prog['month'].max())].groupby('region')[['target_omset','gross_revenue']].sum().reset_index()
    region_summary.to_excel(writer, sheet_name='Region_Summary_Latest', index=False)

print('Exported to', EXPORT_PATH)


Exported to interactive_marketing_reports.xlsx


## Kesimpulan & Rekomendasi (Auto-generated)

Di bagian ini kamu akan menemukan rangkuman otomatis setelah menjalankan semua sel: tren penjualan, produk prioritas, campaign dengan ROI tinggi, dan rekomendasi tindakan 30 hari.

- **Sales Trend:** (Isi setelah melihat grafik) contoh: *omzet cenderung meningkat sejak Q2; perlu jaga momentum lewat kampanye di channel dengan ROI tertinggi.*  
- **Produk & Kategori:** produk top menyumbang porsi besar omzet (**Pareto**). Fokus alokasi stok & promosi pada 10 produk teratas dan kategori margin tinggi.  
- **Promosi & Campaign:** dorong kampanye dengan **ROI > 1** dan lakukan *A/B test* kreatif/channel untuk kampanye yang underperform.  
- **Tim Marketing:** gunakan dashboard capaian untuk *coaching* anggota yang tertinggal; apresiasi top performer untuk menjaga motivasi tim.  
- **Action Plan 30 Hari:**  
  1) Terapkan **review harian** (dashboard + export).  
  2) Prioritaskan **3 produk top** untuk kampanye berikutnya.  
  3) Uji **2 channel promosi** dengan performa terbaik.  
  4) Lakukan **weekly stand-up** dengan tim untuk menutup gap target.

Silakan jalankan seluruh notebook, gunakan widget untuk memilih periode/region yang diinginkan, lalu jalankan sel Ekspor untuk mendapatkan file Excel.